# **Project 2:**

In [9]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from src.Ward import Ward, initialize_wards
from src.Patient import Patient, initialize_patients
from src.simulation import simulation_loop, run_simulations,compute_gradient

## initialization

In [10]:
F_bed_capacity = 30 ##### Number to optimize #####

#patient type from 'A' to 'F'
patient_types = ['A', 'B', 'C', 'D', 'E', 'F']
bed_capacity = np.array([55,40,30,20,20, F_bed_capacity])
arrivals_pr_day = np.array([14.5,11.0,8.0,6.5,5.0, 13.0])
mean_length_of_stay = np.array([2.9,4.0,4.5,1.4,3.9, 2.2])
urgency_points = np.array([7,5,2,10,5, 0])

#reloaction probability
relocation_probability = np.array([
    [0.0, 0.05, 0.10, 0.05, 0.80, 0.00],
    [0.2, 0, 0.50, 0.15, 0.15, 0.00],
    [0.30, 0.20, 0, 0.20, 0.30, 0.00],
    [0.35, 0.30, 0.05, 0, 0.3, 0.00],
    [0.20, 0.10, 0.60 ,0.10, 0, 0.00],
    [0.20, 0.20, 0.20, 0.20, 0.20 ,0]
    ])

#Dataframe containing all the data (indexed by patient type)
df = pd.DataFrame({
    'Patient Type': patient_types,
    'Bed Capacity': bed_capacity,
    'Arrivals per day': arrivals_pr_day,
    'Mean Length of Stay': mean_length_of_stay,
    'Urgency Points': urgency_points
    }).set_index('Patient Type')

In [11]:
#sampling functions. Takes type as input and returns a sample with the corresponding distribution
arrival_interval_function = lambda type: np.random.exponential(1/df["Arrivals per day"][type])
occupancy_time_function = lambda type: np.random.exponential(df["Mean Length of Stay"][type])

# Optimize F beds

Build a simulation model that simulates the patient flow in the hospital as a function of the bed
distribution and the aforementioned parameters

In [12]:
total_time = 31 #simulate 1 month
wards = initialize_wards(df, real_time_tracking = True)
performance = run_simulations(total_time, wards, relocation_probability, arrival_interval_function, occupancy_time_function, n_simulations = 10, verbose = False)
performance

{A Ward with 55 beds and 7 urgency points.: {'Occupied probability': 0.045990135948014085,
  'Estimated admissions': 479.09999999999997,
  'Estimated rejections': 23.7,
  'Estimated relocations': 11.200000000000001},
 B Ward with 40 beds and 5 urgency points.: {'Occupied probability': 0.22719818750366746,
  'Estimated admissions': 282.8,
  'Estimated rejections': 84.10000000000001,
  'Estimated relocations': 53.5},
 C Ward with 30 beds and 2 urgency points.: {'Occupied probability': 0.3497354226276163,
  'Estimated admissions': 209.0,
  'Estimated rejections': 113.0,
  'Estimated relocations': 63.400000000000006},
 D Ward with 20 beds and 10 urgency points.: {'Occupied probability': 0.04020840923734448,
  'Estimated admissions': 244.5,
  'Estimated rejections': 10.5,
  'Estimated relocations': 6.3},
 E Ward with 20 beds and 5 urgency points.: {'Occupied probability': 0.34083478697028763,
  'Estimated admissions': 144.8,
  'Estimated rejections': 74.9,
  'Estimated relocations': 31.7000

In [13]:
occupancy_last_sim = []
rejections_last_sim = []

for ward in wards:
    ward_occupancy = np.array(ward.occupancy)
    ward_rejections = np.array(ward.rejections)
    occupancy_last_sim.append(ward_occupancy[ward_occupancy[:,0] < total_time, :])
    rejections_last_sim.append(ward_rejections[ward_rejections < total_time])

In [14]:
fig = go.Figure()
for i,ward in enumerate(wards):
    occupancy = occupancy_last_sim[i]
    color = 'rgba(' + str(np.random.randint(0,256)) + ',' + str(np.random.randint(0,256)) + ',' + str(np.random.randint(0,256)) + ',1)'
    fig.add_trace(go.Scatter(x = occupancy[:,0], y = occupancy[:,1], mode = 'lines', name = ward.type, line = dict(color = color)))
    #plot upper bound as line for each ward
    fig.add_trace(go.Scatter(x = occupancy[:,0], y = ward.capacity*np.ones(occupancy.shape[0]), mode = 'lines', line = dict(dash = 'dash', color = color), name = ward.type + ' upper bound'))
    #plot rejections as crosses for each ward
    rejections = rejections_last_sim[i]
    fig.add_trace(go.Scatter(x = rejections, y = ward.capacity*np.ones(len(rejections)), mode = 'markers', marker = dict(symbol = 'x', size = 7, color = color, opacity=0.5), name = ward.type + ' rejections'))
fig.update_layout(title = 'Occupancy through time for each simulation in each ward', xaxis_title = 'Time', yaxis_title = 'Occupancy', height = 600)
fig.show()
#save figure as png
fig.write_image("plots/occupancy_through_time.png")

In [15]:
#convert performance to dataframe
df_performance = pd.DataFrame(performance).T
#exclude last row
df_performance = df_performance.iloc[:-1]
#change index
df_performance.index = [ward.type for ward in wards]
df_performance["Estimated lost"] = df_performance["Estimated rejections"] - df_performance["Estimated relocations"]

In [16]:
#create a stacked bar chart of the occupancy of the wards
import plotly.express as px
fig = px.bar(df_performance[["Estimated admissions", "Estimated relocations", "Estimated lost"]])
fig.update_layout(barmode='stack',
                    title='Occupancy of the wards after 1 month',
                    xaxis_title='Ward',
                    yaxis_title='Number of patients')
fig.show()
#save fig in plots
fig.write_image("plots/occupancy.png")

 Create a new ward (F
∗
) in the system and allocate a minimal number of the current bed
resources to the new ward. Ensure that at least 95% of the type F
∗ patients are hospitalized in
Ward F
∗
. Use the ”urgency points” from Table 1 to balance the solution (prioritize wards that
need beds more than other wards).

In [17]:
F_bed_capacity = 0
F_occupied_prob = 1.0
penalties_F = []
F_occupied_probabilities = []
while F_occupied_prob > 0.01:
    #set F bed capacity in the dataframe
    df.loc['F', 'Bed Capacity'] = F_bed_capacity
    wards = initialize_wards(df) #list of wards
    performance = run_simulations(total_time, wards, relocation_probability, arrival_interval_function, occupancy_time_function, n_simulations = 10, verbose = False)
    F_ward = wards[-1]
    F_occupied_prob = performance[F_ward]["Occupied probability"]
    print("Probability of rejection",F_occupied_prob)
    penalties_F.append(performance["Weighted penalty"])
    F_occupied_probabilities.append(F_occupied_prob)
    F_bed_capacity += 1

Probability of rejection 0.9999999999999999
Probability of rejection 0.9662367794757886
Probability of rejection 0.9367319781620882
Probability of rejection 0.8975679576045298
Probability of rejection 0.8730948587236332
Probability of rejection 0.8339200075720529
Probability of rejection 0.7980939919764231
Probability of rejection 0.7566624214030508
Probability of rejection 0.7234054127450056
Probability of rejection 0.706341137162145
Probability of rejection 0.6599420949761025
Probability of rejection 0.6281365963832579
Probability of rejection 0.5815110838961315
Probability of rejection 0.5745158248875047
Probability of rejection 0.543273488481646
Probability of rejection 0.5117212069627192
Probability of rejection 0.4873833702437056
Probability of rejection 0.4369595212549382
Probability of rejection 0.4225948669471058
Probability of rejection 0.36145272107817555
Probability of rejection 0.3602327760641686
Probability of rejection 0.33721835729998284
Probability of rejection 0.28629

In [18]:
#Number of beds required to ensure that 95% of the arrivals in F are admitted
F_bed_capacity_optimal = np.argmax(np.array(F_occupied_probabilities) < 0.05)
print("Number of beds required to ensure that 95% of the arrivals in F are admitted", F_bed_capacity_optimal)

Number of beds required to ensure that 95% of the arrivals in F are admitted 34


In [19]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = np.arange(0, F_bed_capacity), y = penalties_F, name='Weighted penalty'))
fig.update_layout(title = "Penalty and rejection rate as a function of F bed capacity",
                  xaxis_title = "F bed capacity",
                  yaxis_title = "Weighted penalty")
fig.add_trace(go.Scatter(x = np.arange(0, F_bed_capacity), y = F_occupied_probabilities, yaxis = 'y2', name='Rejection probability'))
fig.update_layout(yaxis2 = dict(title = "Rejection probability", overlaying = 'y', side = 'right',
                                ))# Find the index where F_occupied_probabilities is closest to 0.05
index = (np.abs(np.array(F_occupied_probabilities) - 0.05)).argmin()

# Add a point at that index
fig.add_trace(go.Scatter(x=[np.arange(0, F_bed_capacity)[index]], 
                         y=[F_occupied_probabilities[index]], 
                         mode='markers', 
                         marker=dict(size=10, color='red'), 
                         name='5% Rejection'))
fig.show()
#save fig in plots
fig.write_image("plots/penalty_rejection.png")

At some point increasing the number of beds in a ward will not dramatically decrease the penalty.

# Sensitivity analysis
Test the system’s sensitivity to the length-of-stay distribution by replacing the exponential
distribution with the log-normal distribution. Test the new distribution by gradually increasing
the variance (e.g. σ
2
i = 2/µ2
i
, 3/µ2
i
and 4/µ2
i
)

In [20]:
#sampling functions. Takes type as input and returns a sample with the corresponding distribution
#Test the system’s sensitivity to the length-of-stay distribution by replacing the exponential
#distribution with the log-normal distribution. Test the new distribution by gradually increasing
#the variance 

arrival_interval_function = lambda type: np.random.exponential(1/df["Arrivals per day"][type])
occupancy_time_function = lambda var_scale : lambda type: np.random.lognormal(mean = np.log(df["Mean Length of Stay"][type]), sigma = np.sqrt(var_scale/df["Mean Length of Stay"][type]**2))

In [21]:
var_scales = np.linspace(1, 10, 10)
penalties = []
df.loc['F', 'Bed Capacity'] = F_bed_capacity_optimal
for var_scale in var_scales:
    wards = initialize_wards(df) #list of wards
    occupancy_time_function_var = occupancy_time_function(var_scale)
    performance = run_simulations(total_time, wards, relocation_probability, arrival_interval_function, occupancy_time_function_var, n_simulations = 10, verbose = False)
    penalties.append(performance["Weighted penalty"])

In [22]:
#plot the results with PyPlot
fig = go.Figure()
fig.add_trace(go.Scatter(x = var_scales, y = penalties, name='Weighted penalty'))
fig.update_layout(title = "Weighted penalty as variance increases",
                  xaxis_title = "μ^2 * σ^2",
                  yaxis_title = "Weighted penalty")
fig.show()
#save fig in plots
fig.write_image("plots/penalty_variance.png")

Test the system’s sensitivity to the distribution of beds in the hospital.

In [23]:
#gradient of the weighted penalty as a function of the number of beds in each ward
arrival_interval_function = lambda type: np.random.exponential(1/df["Arrivals per day"][type])
occupancy_time_function = lambda type: np.random.exponential(df["Mean Length of Stay"][type])

In [24]:
# Get the gradients for each ward
wards = initialize_wards(df) #list of wards
gradients = compute_gradient(total_time, wards, relocation_probability, arrival_interval_function, occupancy_time_function, n_simulations = 100, verbose = False)

# Create a color array, setting the color to 'red' if the gradient is negative, 'blue' otherwise
colors = ['red' if gradient < 0 else 'blue' for gradient in gradients]

fig = go.Figure()
fig.add_trace(go.Bar(x=[ward.type for ward in wards], y=gradients, marker_color=colors))
fig.update_layout(title="Gradient of the weighted penalty with respect to the number of beds in each ward",
                  xaxis_title="Ward",
                  yaxis_title="Weighted penalty")
fig.show()
#save fig in plots
fig.write_image("plots/sensitivity.png")

valuate the result of increasing the total amount of beds in the system to for instance 170 or
180 beds. Also, what would be the result of having fewer beds